In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define time horizons
time_horizons = [1, 3, 5, 10, 15]

for T in time_horizons:
    print(f"\n🔹 Training Model for T={T} Steps Ahead 🔹")

    # Load prepared multi-step training data
    X_train = np.load(f"/content/drive/MyDrive/mini project/X_train_T{T}.npy")
    y_train = np.load(f"/content/drive/MyDrive/mini project/y_train_T{T}.npy")

    # Define LSTM model
    def build_xlstm_model(input_shape, output_steps):
        model = Sequential([
            LSTM(256, return_sequences=True, activation='tanh', input_shape=input_shape),
            Dropout(0.3),
            LSTM(128, return_sequences=False, activation='tanh'),
            Dropout(0.3),
            Dense(output_steps)  # Predict T steps at once
        ])
        model.compile(optimizer='adam', loss='mse')
        return model

    # Build & Train the model
    model = build_xlstm_model((X_train.shape[1], X_train.shape[2]), output_steps=T)
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

    # Save the trained model for each T value
    model.save(f"/content/drive/MyDrive/mini project/xlstm_model_T{T}.keras")
    print(f"✅ Model for T={T} saved as xlstm_model_T{T}.keras")

print("All models trained and saved successfully!")



🔹 Training Model for T=1 Steps Ahead 🔹


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - loss: 0.0045 - val_loss: 0.0014
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 2.4286e-04 - val_loss: 0.0010
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 2.2424e-04 - val_loss: 0.0016
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 2.1473e-04 - val_loss: 7.6995e-04
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 2.0411e-04 - val_loss: 9.2689e-04
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1.8583e-04 - val_loss: 0.0019
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 2.0040e-04 - val_loss: 6.8343e-04
Epoch 8/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1.8542e-04 - val_loss: 8.6794e-04
Epoch 9/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1.7141e-04 - val_loss: 0.0016
Epoch 10/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1.5851e-04 - val_loss: 5.0178e-04
Epoch 11/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1.6704e-04 - val_loss:

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Load preprocessed data
X = np.load("/content/drive/MyDrive/mini project/X.npy")
y = np.load("/content/drive/MyDrive/mini project/y.npy")

# Define time horizons
time_horizons = [1, 3, 5, 10, 15]

# Function to build xLSTM model
def build_xlstm_model(input_shape, output_steps):
    model = Sequential([
        LSTM(256, return_sequences=True, activation='tanh', input_shape=input_shape),
        Dropout(0.2),
        LSTM(256, return_sequences=False, activation='tanh'),
        Dropout(0.2),
        Dense(output_steps)  # Predict T steps at once
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

for T in time_horizons:
    print(f"\n🔹 Training Model for T={T} Steps Ahead 🔹")

    # Prepare multi-step output labels
    y_train = np.array([y[i:i+T] for i in range(len(y) - T)])
    X_train = X[:len(y_train)]

    # Build model
    model = build_xlstm_model((X_train.shape[1], X_train.shape[2]), output_steps=T)

    # Define learning rate scheduler
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, min_lr=1e-5)

    # Train the model
    epochs = 100 if T <= 5 else 150
    model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_split=0.2, callbacks=[lr_scheduler])

    # Save trained model
    model.save(f"/content/drive/MyDrive/mini project/xLSTM_model_T{T}.keras")
    print(f"✅ Model for T={T} saved as xLSTM_model_T{T}.keras")

print("All models trained and saved successfully!")



🔹 Training Model for T=1 Steps Ahead 🔹


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - loss: 0.0122 - val_loss: 7.7367e-04 - learning_rate: 0.0010
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 5.1163e-04 - val_loss: 7.2779e-04 - learning_rate: 0.0010
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 4.2221e-04 - val_loss: 0.0017 - learning_rate: 0.0010
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 4.1169e-04 - val_loss: 0.0016 - learning_rate: 0.0010
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 4.0232e-04 - val_loss: 0.0010 - learning_rate: 0.0010
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 3.8341e-04 - val_loss: 9.1171e-04 - learning_rate: 0.0010
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 3.6142e-04 - val_loss: 0.0012 - learning_rate: 5.0000e-04
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.1517e-04 - val_loss: 0.0010 - learning_rate: 5.0000e-04
Epoch 9/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.0532e-04 -